In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from pathlib import Path
import plotly.express as px
import plotly as py
import psycopg2
from pyproj import Proj, transform

In [7]:
path = Path.cwd() / 'codepo_gb.gpkg'

In [8]:
path

WindowsPath('c:/Jupyter/Python/Python_Projects/Electricity/codepo_gb.gpkg')

In [9]:
data_path = Path.cwd() / 'Postcode_level_standard_electricity_2015_A_to_L.csv'

In [10]:
df = pd.read_csv(data_path)

In [11]:
df.head()

,POSTCODE,Number of meters,Consumption (kWh),Mean consumption (kWh),Median consumption (kWh)
0,AB10,188,744918.3,3962.331383,3189.9
1,AB10 1AU,24,44997.1,1874.879167,1507.7
2,AB10 1BA,29,61458.4,2119.255172,1832.3
3,AB10 1BB,6,16915.9,2819.316667,2216.2
4,AB10 1FG,25,68685.4,2747.416000,2110.3


In [12]:
gdf = gpd.read_file(path)

In [13]:
gdf.loc[0:5,['Postcode','geometry']]

,Postcode,geometry
0,AB101AB,POINT (394235.000 806529.000)
1,AB101AF,POINT (394181.000 806429.000)
2,AB101AG,POINT (394230.000 806469.000)
3,AB101AH,POINT (394371.000 806359.000)
4,AB101AL,POINT (394296.000 806581.000)
5,AB101AN,POINT (394367.000 806541.000)


In [14]:
gdf.describe()

,Positional_quality_indicator
count,1.714213e+06
mean,1.013526e+01
std,2.547296e+00
min,1.000000e+01
25%,1.000000e+01
50%,1.000000e+01
75%,1.000000e+01
max,9.000000e+01


In [15]:
df.columns

Index(['POSTCODE', 'Number of meters', 'Consumption (kWh)',
       'Mean consumption (kWh)', 'Median consumption (kWh)'],
      dtype='object')

In [16]:
cols = ['Postcode', 'meters', 'consumption', 'mean_consumption', 'median_consumption']
df.columns = cols
df.columns

Index(['Postcode', 'meters', 'consumption', 'mean_consumption',
       'median_consumption'],
      dtype='object')

In [17]:
gpd_edit = gdf.loc[:,['Postcode', 'geometry']]

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Postcode            500000 non-null  object 
 1   meters              500000 non-null  int64  
 2   consumption         500000 non-null  float64
 3   mean_consumption    500000 non-null  float64
 4   median_consumption  500000 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 19.1+ MB


In [19]:
gpd_edit.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1714213 entries, 0 to 1714212
Data columns (total 2 columns):
 #   Column    Dtype   
---  ------    -----   
 0   Postcode  object  
 1   geometry  geometry
dtypes: geometry(1), object(1)
memory usage: 26.2+ MB


In [20]:
df.set_index('Postcode').join(gpd_edit.set_index('Postcode'))

,meters,consumption,mean_consumption,median_consumption,geometry
Postcode,,,,,
AB10,188,744918.3,3962.331383,3189.90,None
AB10 1AU,24,44997.1,1874.879167,1507.70,None
AB10 1BA,29,61458.4,2119.255172,1832.30,None
AB10 1BB,6,16915.9,2819.316667,2216.20,None
AB10 1FG,25,68685.4,2747.416000,2110.30,None
...,...,...,...,...,...
L17 8TR,65,190108.8,2924.750769,2014.90,POINT (337029.000 387831.000)
L17 8TS,35,86484.4,2470.982857,1931.10,POINT (337017.000 387733.000)
L17 8TT,32,67227.4,2100.856250,1648.40,POINT (337087.000 387663.000)


In [21]:
merged = df.set_index('Postcode').join(gpd_edit.set_index('Postcode'))

In [22]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, AB10 to L17 8TX
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   meters              500000 non-null  int64   
 1   consumption         500000 non-null  float64 
 2   mean_consumption    500000 non-null  float64 
 3   median_consumption  500000 non-null  float64 
 4   geometry            273444 non-null  geometry
dtypes: float64(3), geometry(1), int64(1)
memory usage: 39.0+ MB


In [23]:
rows_to_drop = merged[merged['geometry'] == None].index

In [24]:
merged.drop(rows_to_drop, inplace=True)

In [25]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273444 entries, AL1 1AJ to L17 8TX
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   meters              273444 non-null  int64   
 1   consumption         273444 non-null  float64 
 2   mean_consumption    273444 non-null  float64 
 3   median_consumption  273444 non-null  float64 
 4   geometry            273444 non-null  geometry
dtypes: float64(3), geometry(1), int64(1)
memory usage: 12.5+ MB


In [26]:
merged.head()

,meters,consumption,mean_consumption,median_consumption,geometry
Postcode,,,,,
AL1 1AJ,50,146369.9,2927.398000,2557.80,POINT (515491.000 206410.000)
AL1 1AR,21,51268.8,2441.371429,2226.80,POINT (516270.000 205897.000)
AL1 1AS,13,42842.1,3295.546154,2712.80,POINT (515005.000 206908.000)
AL1 1AT,8,30174.6,3771.825000,4238.25,POINT (516131.000 206148.000)
AL1 1BX,16,81580.8,5098.800000,4680.40,POINT (514619.000 206893.000)


In [27]:
merged['eastings'] = 0

In [28]:
merged['northings'] = 0

In [29]:
merged.head()

,meters,consumption,mean_consumption,median_consumption,geometry,eastings,northings
Postcode,,,,,,,
AL1 1AJ,50,146369.9,2927.398000,2557.80,POINT (515491.000 206410.000),0,0
AL1 1AR,21,51268.8,2441.371429,2226.80,POINT (516270.000 205897.000),0,0
AL1 1AS,13,42842.1,3295.546154,2712.80,POINT (515005.000 206908.000),0,0
AL1 1AT,8,30174.6,3771.825000,4238.25,POINT (516131.000 206148.000),0,0
AL1 1BX,16,81580.8,5098.800000,4680.40,POINT (514619.000 206893.000),0,0


In [30]:
merged.geometry.loc['AL1 1AJ'].x

515491.0

In [31]:
merged

,meters,consumption,mean_consumption,median_consumption,geometry,eastings,northings
Postcode,,,,,,,
AL1 1AJ,50,146369.9,2927.398000,2557.80,POINT (515491.000 206410.000),0,0
AL1 1AR,21,51268.8,2441.371429,2226.80,POINT (516270.000 205897.000),0,0
AL1 1AS,13,42842.1,3295.546154,2712.80,POINT (515005.000 206908.000),0,0
AL1 1AT,8,30174.6,3771.825000,4238.25,POINT (516131.000 206148.000),0,0
AL1 1BX,16,81580.8,5098.800000,4680.40,POINT (514619.000 206893.000),0,0
...,...,...,...,...,...,...,...
L17 8TR,65,190108.8,2924.750769,2014.90,POINT (337029.000 387831.000),0,0
L17 8TS,35,86484.4,2470.982857,1931.10,POINT (337017.000 387733.000),0,0
L17 8TT,32,67227.4,2100.856250,1648.40,POINT (337087.000 387663.000),0,0


In [32]:
eastings = merged.apply(
            lambda row: (row['geometry'].x),
            axis = 1)

In [33]:
northings = merged.apply(
            lambda row: (row['geometry'].y),
            axis = 1)

In [34]:
merged['eastings'] = eastings

In [35]:
merged['northings'] = northings

In [36]:
merged

,meters,consumption,mean_consumption,median_consumption,geometry,eastings,northings
Postcode,,,,,,,
AL1 1AJ,50,146369.9,2927.398000,2557.80,POINT (515491.000 206410.000),515491.0,206410.0
AL1 1AR,21,51268.8,2441.371429,2226.80,POINT (516270.000 205897.000),516270.0,205897.0
AL1 1AS,13,42842.1,3295.546154,2712.80,POINT (515005.000 206908.000),515005.0,206908.0
AL1 1AT,8,30174.6,3771.825000,4238.25,POINT (516131.000 206148.000),516131.0,206148.0
AL1 1BX,16,81580.8,5098.800000,4680.40,POINT (514619.000 206893.000),514619.0,206893.0
...,...,...,...,...,...,...,...
L17 8TR,65,190108.8,2924.750769,2014.90,POINT (337029.000 387831.000),337029.0,387831.0
L17 8TS,35,86484.4,2470.982857,1931.10,POINT (337017.000 387733.000),337017.0,387733.0
L17 8TT,32,67227.4,2100.856250,1648.40,POINT (337087.000 387663.000),337087.0,387663.0


In [37]:

v84 = Proj(proj="latlong",towgs84="0,0,0",ellps="WGS84")
v36 = Proj(proj="latlong", k=0.9996012717, ellps="airy",
        towgs84="446.448,-125.157,542.060,0.1502,0.2470,0.8421,-20.4894")
vgrid = Proj(init="world:bng")


def vectorized_converted(df):

    eastings = df['eastings'].values
    northings = df['northings'].values

    vlon36, vlat36 = vgrid(eastings, northings, inverse=True)

    converted = transform(v36, v84, vlon36, vlat36)

    df['longitude'] = converted[0]
    df['latitude'] = converted[1]

    return df

C:\Program Files\Python39\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Program Files\Python39\lib\site-packages\pyproj\crs\crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [38]:
vectorized_converted(merged)

<ipython-input-37-b848c86c72bc>:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  converted = transform(v36, v84, vlon36, vlat36)


,meters,consumption,mean_consumption,median_consumption,geometry,eastings,northings,longitude,latitude
Postcode,,,,,,,,,
AL1 1AJ,50,146369.9,2927.398000,2557.80,POINT (515491.000 206410.000),515491.0,206410.0,-0.328599,51.744498
AL1 1AR,21,51268.8,2441.371429,2226.80,POINT (516270.000 205897.000),516270.0,205897.0,-0.317492,51.739727
AL1 1AS,13,42842.1,3295.546154,2712.80,POINT (515005.000 206908.000),515005.0,206908.0,-0.335471,51.749073
AL1 1AT,8,30174.6,3771.825000,4238.25,POINT (516131.000 206148.000),516131.0,206148.0,-0.319421,51.742011
AL1 1BX,16,81580.8,5098.800000,4680.40,POINT (514619.000 206893.000),514619.0,206893.0,-0.341065,51.749018
...,...,...,...,...,...,...,...,...,...
L17 8TR,65,190108.8,2924.750769,2014.90,POINT (337029.000 387831.000),337029.0,387831.0,-2.948180,53.383554
L17 8TS,35,86484.4,2470.982857,1931.10,POINT (337017.000 387733.000),337017.0,387733.0,-2.948341,53.382671
L17 8TT,32,67227.4,2100.856250,1648.40,POINT (337087.000 387663.000),337087.0,387663.0,-2.947275,53.382051


In [39]:
merged = merged.drop(columns=['meters', 'consumption', 'geometry', 'northings', 'eastings'])
merged = merged.reset_index()

In [40]:
merged

,Postcode,mean_consumption,median_consumption,longitude,latitude
0,AL1 1AJ,2927.398000,2557.80,-0.328599,51.744498
1,AL1 1AR,2441.371429,2226.80,-0.317492,51.739727
2,AL1 1AS,3295.546154,2712.80,-0.335471,51.749073
3,AL1 1AT,3771.825000,4238.25,-0.319421,51.742011
4,AL1 1BX,5098.800000,4680.40,-0.341065,51.749018
...,...,...,...,...,...
273439,L17 8TR,2924.750769,2014.90,-2.948180,53.383554
273440,L17 8TS,2470.982857,1931.10,-2.948341,53.382671
273441,L17 8TT,2100.856250,1648.40,-2.947275,53.382051
273442,L17 8TU,2673.111364,2119.30,-2.948850,53.382568


In [41]:
merged.set_index('Postcode')
merged

,Postcode,mean_consumption,median_consumption,longitude,latitude
0,AL1 1AJ,2927.398000,2557.80,-0.328599,51.744498
1,AL1 1AR,2441.371429,2226.80,-0.317492,51.739727
2,AL1 1AS,3295.546154,2712.80,-0.335471,51.749073
3,AL1 1AT,3771.825000,4238.25,-0.319421,51.742011
4,AL1 1BX,5098.800000,4680.40,-0.341065,51.749018
...,...,...,...,...,...
273439,L17 8TR,2924.750769,2014.90,-2.948180,53.383554
273440,L17 8TS,2470.982857,1931.10,-2.948341,53.382671
273441,L17 8TT,2100.856250,1648.40,-2.947275,53.382051
273442,L17 8TU,2673.111364,2119.30,-2.948850,53.382568


In [42]:
#Create data into .csv file to feed into postgresql
merged.to_csv (r'C:\temp\postcode.csv', index = False, header=True)

In [3]:
#connect to psql after postcode table is filled with the data from the .csv file
#use this as the new merged dataset

try:
    conn=psycopg2.connect("dbname='Datasets' user='postgres' password='Admin123'")
except:
    print ("I am unable to connect to the database.")


cur = conn.cursor()
try:
    cur.execute("""SELECT postcode AS Postcode
                        ,mean_consumption AS mean_consummption
                        ,median_consumption AS median_consummption
                        ,longitude AS longitude
                        ,latitude AS latitude
                FROM postcode
                --LIMIT 100""")
except:
    print ("I can't SELECT from postcode")

rows = cur.fetchall()

#print(rows)

merged = pd.DataFrame(rows)

#print(merged)

merged.set_index(0)

#print(merged)

conn.close()

In [44]:
#change the column names to something useful
merged.columns =['postcode', 'mean_consumption', 'median_consumption', 'longitude', 'latitude']
merged

,postcode,mean_consumption,median_consumption,longitude,latitude
0,AL1 1AJ,2927.3980,2557.80,-0.328599,51.744500
1,AL1 1AR,2441.3713,2226.80,-0.317492,51.739727
2,AL1 1AS,3295.5461,2712.80,-0.335471,51.749073
3,AL1 1AT,3771.8250,4238.25,-0.319421,51.742012
4,AL1 1BX,5098.8000,4680.40,-0.341065,51.749016
...,...,...,...,...,...
273439,L17 8TR,2924.7507,2014.90,-2.948180,53.383553
273440,L17 8TS,2470.9830,1931.10,-2.948341,53.382670
273441,L17 8TT,2100.8562,1648.40,-2.947274,53.382050
273442,L17 8TU,2673.1113,2119.30,-2.948850,53.382570


In [45]:
#Set median_consumption from 
merged['median_consumption'].median()
merged

,postcode,mean_consumption,median_consumption,longitude,latitude
0,AL1 1AJ,2927.3980,2557.80,-0.328599,51.744500
1,AL1 1AR,2441.3713,2226.80,-0.317492,51.739727
2,AL1 1AS,3295.5461,2712.80,-0.335471,51.749073
3,AL1 1AT,3771.8250,4238.25,-0.319421,51.742012
4,AL1 1BX,5098.8000,4680.40,-0.341065,51.749016
...,...,...,...,...,...
273439,L17 8TR,2924.7507,2014.90,-2.948180,53.383553
273440,L17 8TS,2470.9830,1931.10,-2.948341,53.382670
273441,L17 8TT,2100.8562,1648.40,-2.947274,53.382050
273442,L17 8TU,2673.1113,2119.30,-2.948850,53.382570


In [46]:
merged['median_consumption'].quantile(0.99)
merged

,postcode,mean_consumption,median_consumption,longitude,latitude
0,AL1 1AJ,2927.3980,2557.80,-0.328599,51.744500
1,AL1 1AR,2441.3713,2226.80,-0.317492,51.739727
2,AL1 1AS,3295.5461,2712.80,-0.335471,51.749073
3,AL1 1AT,3771.8250,4238.25,-0.319421,51.742012
4,AL1 1BX,5098.8000,4680.40,-0.341065,51.749016
...,...,...,...,...,...
273439,L17 8TR,2924.7507,2014.90,-2.948180,53.383553
273440,L17 8TS,2470.9830,1931.10,-2.948341,53.382670
273441,L17 8TT,2100.8562,1648.40,-2.947274,53.382050
273442,L17 8TU,2673.1113,2119.30,-2.948850,53.382570


In [47]:
high_consumption = merged[merged['median_consumption'] > merged['median_consumption'].quantile(0.99)]

In [48]:
high_consumption.describe()

,mean_consumption,median_consumption,longitude,latitude
count,2735.000000,2735.000000,2735.000000,2735.000000
mean,10475.785842,9516.376700,-1.384811,52.582181
std,3025.342832,2156.958217,1.468519,1.577212
min,5547.460000,7672.100000,-7.374790,50.584038
25%,8481.341250,8134.400000,-2.413431,51.467087
50%,9706.440000,8823.700000,-1.483927,51.801517
75%,11634.982500,10062.600000,-0.176804,53.648735
max,34234.840000,31487.050000,1.399202,58.340336


In [49]:
high_consumption

,postcode,mean_consumption,median_consumption,longitude,latitude
290,AL1 3GJ,9507.0370,8894.55,-0.329987,51.754400
296,AL1 3GS,17137.4430,13326.30,-0.329852,51.754120
298,AL1 3GU,10964.0500,10445.20,-0.329472,51.753807
303,AL1 3NG,10161.2330,9167.85,-0.330175,51.754005
314,AL1 3PP,7799.0000,7849.70,-0.326373,51.756912
...,...,...,...,...,...
270568,L10 8LE,8338.2000,7884.80,-2.941991,53.481113
271242,L12 1LJ,10417.5400,9877.40,-2.913509,53.426296
271272,L12 2BA,8307.5930,8270.80,-2.901420,53.421120
272341,L14 7AD,6828.2600,8367.10,-2.885633,53.410900


In [50]:
#Uncomment to see plot. Plot to big on initial boot

#fig = px.scatter_mapbox(merged, lat="latitude", lon= "longitude", color="median_consumption", zoom=5, mapbox_style='open-street-map', 
#                        hover_name='postcode', hover_data=['median_consumption'])

#fig.show()

In [51]:
#Plotting the highest consumptions on the map
#fig = px.scatter_mapbox(high_consumption, lat="latitude", lon= "longitude", color="median_consumption", zoom=5, mapbox_style='open-street-map', 
#                        hover_name='postcode', hover_data=['median_consumption'])

#fig.show()